## Vocab and Label

In [1]:
from utils.tokenizer import Vocabulary
vocab = Vocabulary.load("./data/process/vocab.txt")
label_list = ['财经', '彩票', '房产', '股票', '家居', '教育', '科技', '社会', '时尚', '时政', '体育', '星座', '游戏', '娱乐']

## Data

In [2]:
from utils.dataset import NewsDataset, collate_fn
from torch.utils.data import DataLoader
train_dataset = NewsDataset("./data/process/train.txt")
val_dataset = NewsDataset("./data/process/val.txt")
batch_size = 4
train_loader = DataLoader(train_dataset, batch_size=batch_size, collate_fn=collate_fn, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, collate_fn=collate_fn, shuffle=False)

In [3]:
print('=============train_loader =============')
for data, label in train_loader:
    print(data)
    print(data.shape)
    print(label)
    print(label.shape)
    break

=============train_loader =============
tensor([[ 941,  343, 1940,   53,   49, 3476, 2090, 2144, 1842,  225, 1005,   51,
           74,   61,   23,   18, 1646,   11,   48,    9,   12,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1],
        [  23,  166,   52, 2274,  145,   10,  589,  364,  963, 1131,  178,  986,
         2193,  336,   13, 2274,  101,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1],
        [1230,   49, 2007,   73,  374,  659,  354,   33, 1431,    2,   74,  374,
         1276, 1421,  488, 2000, 2008,  113,  136,   83,  327,  131,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1],
        [   6,    3,    4,    3,   17,  116,  313,   15,  251,  178,  824,   67,
          283,  136,  376,   52,  111,   11,    9,   12,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1]])
torch.Size([4, 32])
tensor([ 7,  9, 13,  5])
torch.Size([4])


## Model

In [4]:
from models.rnn import RNN
vocab_size = len(vocab)
embedding_dim = 128
hidden_dim = 128
num_layers = 1
num_classes = len(label_list)
model = RNN(vocab_size, embedding_dim, hidden_dim, num_layers, num_classes)

In [5]:
import torch

x = torch.LongTensor([
    [1,2,3,4,5,6,7,8,9,10,11,12,13,19],
    [1,2,3,4,5,6,7,8,9,10,11,12,13,19],
])

from torchinfo import summary
summary(model, input_data=x)

Layer (type:depth-idx)                   Output Shape              Param #
RNN                                      [2, 14]                   --
├─Embedding: 1-1                         [2, 14, 128]              672,128
├─LSTM: 1-2                              [2, 14, 128]              132,096
├─Linear: 1-3                            [2, 14]                   1,806
Total params: 806,030
Trainable params: 806,030
Non-trainable params: 0
Total mult-adds (M): 5.05
Input size (MB): 0.00
Forward/backward pass size (MB): 0.06
Params size (MB): 3.22
Estimated Total Size (MB): 3.28

## Train

In [6]:
def score(model, dataloader, device):
    y_true = []
    y_pred = []
    with torch.no_grad():
        for x, y in dataloader:
            x = x.to(device)
            y = y.to(device)
            out = model(x)
            y_pred.extend(torch.argmax(out, dim=-1).tolist())
            y_true.extend(y.tolist())
    return y_true, y_pred

In [7]:
import torch.nn as nn
import torch.optim as optim
from utils.animator import Animator

lr = 0.001
num_epochs = 10
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs)
# ani = Animator()

for epoch in range(num_epochs):
    for i, (data, labels) in enumerate(train_loader):
        data = data.to(device)
        labels = labels.to(device)

        outputs = model(data)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 20 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                .format(epoch+1, num_epochs, i+1, len(train_loader), loss.item()))
            
            # y_true, y_pred = score(model, val_loader, device)
            # accuracy = sum([1 if y_true[i] == y_pred[i] else 0 for i in range(len(y_true))]) / len(y_true)
            # inter = i / len(train_loader)
            # ani.ax.plot(epoch + inter, loss.item(), 'r.', ms=10)
            # ani.ax.plot(epoch + inter, accuracy, 'b.', ms=10)
            # ani.ax.set_xlabel('epoch')
            # ani.ax.set_ylabel('loss')
            # ani.ax.set_title('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}'
            #                  .format(epoch+1, num_epochs, i+1, len(train_loader), loss.item(), accuracy))
            # ani.render(0.01)

    y_true, y_pred = score(model, val_loader, device)
    accuracy = sum([1 if y_true[i] == y_pred[i] else 0 for i in range(len(y_true))]) / len(y_true)
    print('Epoch [{}/{}], Accuracy: {:.2f}'
        .format(epoch+1, num_epochs, accuracy))
    

    scheduler.step()

# ani.close()

Epoch [1/10], Step [20/23515], Loss: 2.0458
Epoch [1/10], Step [40/23515], Loss: 1.8403
Epoch [1/10], Step [60/23515], Loss: 3.7955
Epoch [1/10], Step [80/23515], Loss: 2.3585
Epoch [1/10], Step [100/23515], Loss: 1.7105
Epoch [1/10], Step [120/23515], Loss: 1.8702
Epoch [1/10], Step [140/23515], Loss: 2.6521
Epoch [1/10], Step [160/23515], Loss: 2.5443
Epoch [1/10], Step [180/23515], Loss: 2.6538
Epoch [1/10], Step [200/23515], Loss: 2.2453
Epoch [1/10], Step [220/23515], Loss: 1.9674
Epoch [1/10], Step [240/23515], Loss: 2.2733
Epoch [1/10], Step [260/23515], Loss: 2.3976
Epoch [1/10], Step [280/23515], Loss: 2.0486
Epoch [1/10], Step [300/23515], Loss: 2.1470
Epoch [1/10], Step [320/23515], Loss: 2.3423
Epoch [1/10], Step [340/23515], Loss: 2.3574
Epoch [1/10], Step [360/23515], Loss: 2.5475
Epoch [1/10], Step [380/23515], Loss: 2.2412
Epoch [1/10], Step [400/23515], Loss: 2.7647
Epoch [1/10], Step [420/23515], Loss: 2.1533
Epoch [1/10], Step [440/23515], Loss: 1.9607
Epoch [1/10], 

KeyboardInterrupt: 